In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head/

/content/drive/.shortcut-targets-by-id/1gvgdEyQQFFN43xnL2_DdI4rUtMI5gnmU/transformer-drg-style-transfer-master/data/yelp/processed_files_with_t5_with_best_head


In [3]:
from tqdm import tqdm, trange
import numpy as np

In [14]:
def process_file(input_file, output_file):
    """
    The function process the data files for Delete & Generate and convert
    it for the Delete, Retrieve and Generate training by separating the content
    and attributes. It includes all the attribure words.
    
    Input_file: string : Path of the input file
    Output_file: string : Path of the output file 
    """
    max_atts=10
    with open(input_file) as fp:
        data = fp.read().splitlines()
    with open (output_file,"w+") as out_fp:
        for x in tqdm(data):
            if x == "":
              continue
            temp = x.split("<START>")
            #print(temp)
            con = temp[0].replace("<POS>","").replace("<NEG>","").replace("<CON_START>","")
            sen = temp[1].replace("<END>","")
            lt1 = con.split()
            lt2 = sen.split()
            att_tokens = [z for z in lt2 if z not in lt1]
            if len(att_tokens) > max_atts:
                max_atts = len(att_tokens)
            att_words = " ".join(att_tokens)
            out_str = "<ATTR_WORDS> " + att_words + " <CON_START> " + con.strip() + " <START> " + sen.strip() + " <END>" + "\n"
            out_fp.write(out_str)
    

In [15]:
def process_file_v1(input_file, output_file):
    """
    The function process the data files for Delete & Generate and convert
    it for the Delete, Retrieve and Generate training by separating the content
    and attributes. It randomly picks 70% of the attributes only to make the generation
    more realistic instead of just filling the blanks, which helps while generating
    sentences for test cases.
    
    Input_file: string : Path of the input file
    Output_file: string : Path of the output file 
    """
    with open(input_file) as fp:
        data = fp.read().splitlines()
    with open (output_file,"w") as out_fp:
        for x in tqdm(data):
            if x == "":
              continue
            temp = x.split("<START>")
            #print(temp)
            con = temp[0].replace("<POS>","").replace("<NEG>","").replace("<CON_START>","")
            sen = temp[1].replace("<END>","")
            lt1 = con.split()
            lt2 = sen.split()
            att_words = [z for z in lt2 if z not in lt1]
            #print(att_words)
            
            if len(att_words) > 2:
                indx = np.array(list(range(len(att_words))))
                np.random.shuffle(indx)
                att_words = " ".join([att_words[indx[k]] for k in range(int(0.7 * len(att_words)))])
            else: # If attributes less than 2 then keep all the attributes
                att_words = " ".join(att_words)
            #print(att_words)
            out_str = "<ATTR_WORDS> " + att_words + " <CON_START> " + con.strip() + " <START> " + sen.strip() + " <END>" + "\n"
            out_fp.write(out_str)

In [16]:
process_file("sentiment.train", "sentiment_train_all_attrs.txt")
process_file("sentiment.train.1","sentiment_train_1_all_attrs.txt")
process_file("sentiment.train.0","sentiment_train_0_all_attrs.txt")

100%|██████████| 177218/177218 [00:01<00:00, 109241.52it/s]


In [ ]:
%cd /content/drive/MyDrive/transformer-drg-style-transfer-master/data/

/content/drive/MyDrive/transformer-drg-style-transfer-master/data


In [20]:
process_file("sentiment.test","sentiment_test_all_attrs.txt")
process_file("sentiment.test.1","sentiment_test_1_all_attrs.txt")
process_file("sentiment.test.0","sentiment_test_0_all_attrs.txt")

100%|██████████| 500/500 [00:00<00:00, 118449.70it/s]


In [21]:
process_file("sentiment.dev","sentiment_dev_all_attrs.txt")
process_file("sentiment.dev.1","sentiment_dev_1_all_attrs.txt")
process_file("sentiment.dev.0","sentiment_dev_0_all_attrs.txt")

100%|██████████| 2000/2000 [00:00<00:00, 125033.28it/s]


In [22]:
process_file_v1("sentiment.train","sentiment_train.txt")
process_file_v1("sentiment.train.0","sentiment_train_1.txt")
process_file_v1("sentiment.train.1","sentiment_train_0.txt")

100%|██████████| 266041/266041 [00:02<00:00, 92878.85it/s] 


In [23]:
process_file_v1("sentiment.test","sentiment_test.txt")
process_file_v1("sentiment.test.1","sentiment_test_1.txt")
process_file_v1("sentiment.test.0","sentiment_test_0.txt")

100%|██████████| 500/500 [00:00<00:00, 70613.56it/s]


In [24]:
process_file_v1("sentiment.dev","sentiment_dev.txt")
process_file_v1("sentiment.dev.1","sentiment_dev_1.txt")
process_file_v1("sentiment.dev.0","sentiment_dev_0.txt")

100%|██████████| 2000/2000 [00:00<00:00, 112827.45it/s]
